# ** Gender and Age Prediction CNN Model**

# **Import Modules**

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 

from tqdm.notebook import tqdm 
warnings.filterwarnings('ignore') 

%matplotlib inline 

import tensorflow as tf 
from keras.preprocessing import image 
from keras.preprocessing.image import load_img 
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Input 

# **Load the Dataset**

In [ ]:
# the dataset link on kaggle 
# https://www.kaggle.com/datasets/jangedoo/utkface-new?rvi=1 

BASE_DIR = "F:\\A_DATASETS\\NNs\\images gender_age_ethnicity\\UTKFace\\"

In [ ]:
# labels ---> age, gender, ethnicity 
image_paths = [] 
age_labels = []
gender_labels = [] 

for filename in tqdm(os.listdir(BASE_DIR)) :
    image_path = os.path.join(BASE_DIR, filename) 
    
    temp = filename.split('_')
    age = int(temp[0]) 
    gender = int(temp[1]) 
    
    image_paths.append(image_path)
    age_labels.append(age) 
    gender_labels.append(gender) 
    
    


In [ ]:
gender_labels[: 20] 

In [ ]:
# Conver to dataframe 

df = pd.DataFrame() 

df['image'] =  image_paths
df['age'] = age_labels         # 0 ===> male , 1 ===> female 
df['gender'] = gender_labels

df.head()

# ---------- Shuffling the data(images) -------------
# replace df with df_shuffled

# df_shuffled = df.sample(frac=1).reset_index(drop=True)
# df_shuffled[0:10]             




In [ ]:
# map labels for gender (gender dictionary) 

gender_dict = {0:'Male', 1:'Female'} 


# Exploratory Data Analysis

In [ ]:
from PIL import Image 
img = Image.open(df['image'][960]) 

plt.axis('off') 

plt.imshow(img) ;


In [ ]:
# show age distribution
sns.distplot( df['age'])

In [ ]:
# sns.countplot(df['gender'])     ------> doesn't work 
plt.plot(df['gender'][:10])


In [ ]:
print('Males number : ',gender_labels.count(0)) 
print('Females number : ',gender_labels.count(1)) 


***Display grid of images***

In [ ]:
plt.figure(figsize=(15,15)) 
files = df.iloc[0:16]

for index, file, age, gender in files.itertuples() :
    plt.subplot(4, 4, index+1) 
    
    img = load_img(file) 
    img = np.array(img) 
    
    plt.imshow(img) 
    plt.title(f'Age: {age}  Gender: {gender_dict[gender]}') 
    plt.axis('off') 
    


# **Feature Extraction**

In [ ]:
def extract_features(images) :
    features = [] 
    
    for image in tqdm(images) :
        img = load_img(image, color_mode='grayscale') # Convert image to grayscale
        img = img.resize((128, 128))                    #, Image.ANTIALIAS
        img = np.array(img)
         
        features.append(img) 
        
    
    features = np.array(features) 
    features = features.reshape(len(features), 128, 128, 1)  # Reshape for CNN input
    
    
    return features  # Return the extracted features array

In [ ]:
x = extract_features(df['image'])

In [ ]:
x.shape

In [ ]:
x = x/255.0 

In [ ]:
y_gender = np.array(df['gender'])
y_age = np.array(df['age'])

In [ ]:
input_shape = (128, 128 ,1)     

# **Model Creation**

In [ ]:
inputs = Input(input_shape)
# ----------------------- Convolutional Layers -------------------
conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)

conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxp_1)
maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)

conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(maxp_2)
maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)

conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu')(maxp_3)
maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)

flatten = Flatten()(maxp_4)

# Fully connected layers with Dropout
dense_1 = Dense(256, activation='relu')(flatten)
dropout_1 = Dropout(0.3)(dense_1)

dense_2 = Dense(256, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(dense_2)

# Output layers with appropriate activation functions
output_1 = Dense(1, activation='sigmoid', name='gender_out')(dropout_1)
output_2 = Dense(1, activation='linear', name='age_out')(dropout_2)

# -----------------------------------------------------------
model = Model(inputs=[inputs], outputs=[output_1, output_2])


***Model compilation***

In [ ]:
from keras.optimizers import Adam

# Model Compilation with Adam optimizer and specified learning rate

optimizer = Adam(learning_rate=0.001)             

model.compile(loss={'gender_out': 'binary_crossentropy', 'age_out': 'mse'}, 
              optimizer=optimizer, 
              metrics={'gender_out': 'accuracy', 'age_out': 'accuracy'})


Plotting the model 

In [ ]:
# plot the model 
# from tensorflow.keras.utils import plot_model
# plot_model(model)

model.summary() 

***Train the model***

In [ ]:
from keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model Fitting with early stopping ------> , callbacks=[early_stopping] 

history = model.fit(x=x, y=[y_gender, y_age], batch_size=32, epochs=40, validation_split=0.2 ) 


***Saving the model***

In [ ]:
model.save('the_last_trained_model.keras')

## Plot the results

In [ ]:
# Plotting for the gender
acc = history.history['gender_out_accuracy']
val_acc = history.history['val_gender_out_accuracy']
epochs = range(len(acc)) 

plt.plot(epochs, acc, 'b', label='Training Accuracy') 
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy') 
plt.title('Accuracy Graph') 
plt.legend()
plt.figure() 


loss = history.history['loss']
val_loss = history.history['val_loss']
# epochs = range(len(loss)) 

plt.plot(epochs, loss, 'b', label='Training Loss') 
plt.plot(epochs, val_loss, 'r', label='Validation Loss') 
plt.title('Loss Graph') 
plt.legend()
plt.figure() 

plt.show()


In [ ]:
# Plotting for the age
acc = history.history['age_out_accuracy']
val_acc = history.history['val_age_out_accuracy']
epochs = range(len(acc)) 

plt.plot(epochs, acc, 'b', label='Training Accuracy') 
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy') 
plt.title('Accuracy Graph') 
plt.legend()
plt.figure() 

print(f'the final acc result: { acc[-1] }  \nand the final val acc: { val_acc[-1] }')

# loss = history.history['age_out_loss']
# val_loss = history.history['val_age_out_loss']

# plt.plot(epochs, loss, 'b', label='Training Loss') 
# plt.plot(epochs, val_loss, 'r', label='Validation Loss') 
# plt.title('Loss Graph') 
# plt.legend()
# plt.figure() 

# plt.show()


**Loading the model**

In [ ]:
from keras.models import load_model

# Load the trained model
model = load_model('the_best_trained_model.keras')  

# Prediction with Test Data

In [ ]:
# image_index = 960
image_index = np.random.randint(0, len(x))

# print("==========================[ ORIGINAL ]===============================")
# print(f'Gender ----> ((( {gender_dict[y_gender[image_index]]} )))    \t\t\t Age ----> ((( {y_age[image_index]} )))\n' )

# Predicted from model 
pred = model.predict(x[image_index].reshape(1, 128, 128, 1))

pred_gender = gender_dict[round(pred[0][0][0])]
pred_age = round(pred[1][0][0]) 

# print("\n=====================[ PREDICTED Result ]============================")
# print(f"Gender ----> ((( {pred_gender} )))         the Age is around ----> ((( {pred_age} )))")

plt.title(f"Original Gender: ((( {gender_dict[y_gender[image_index]]} )))   original Age: ((( {y_age[image_index]} )))\n\nPredicted Gender: (( {pred_gender} ))  Predicted Age:(( {pred_age} ))")

plt.imshow(x[image_index].reshape(128,128)) ;
plt.axis('off') ;

print("image index = ",image_index)

# **Testing** predicting image out of the dataset 

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Function to preprocess an image for prediction
def preprocess_image(image_path):
    img = Image.open(image_path).convert('L')  # Convert image to grayscale
    img = img.resize((128, 128))
    img = np.array(img)
    img = img.reshape(1, 128, 128, 1)  # Reshape for CNN input
    img = img / 255.0  # Normalize pixel values
    return img

# Function to predict gender and age from an image path
def predict_gender_and_age(image_path, model):
    preprocessed_img = preprocess_image(image_path)
    pred = model.predict(preprocessed_img)
    gender_dict = {0: 'Male', 1: 'Female'}
    gender_pred = gender_dict[round(pred[0][0][0])]
    age_pred = round(pred[1][0][0])
    return gender_pred, age_pred

# Example usage
image_path = r"C:\Users\hp\Downloads\download.jpeg"
predicted_gender, predicted_age = predict_gender_and_age(image_path, model)

# Plot the image
img = Image.open(image_path)
plt.imshow(img, cmap='gray')
plt.title(f"Predicted Gender: {predicted_gender}, Predicted Age: {predicted_age}")
plt.axis('off')
plt.show()


# **Modifications**

In [ ]:
# from keras.optimizers import Adam

# inputs = Input(input_shape)
# # ----------------------- Convolutional Layers -------------------
# conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(inputs)
# maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)

# conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(maxp_1)
# maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)

# conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(maxp_2)
# maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)

# conv_4 = Conv2D(256, kernel_size=(3, 3), activation='relu')(maxp_3)
# maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)

# flatten = Flatten()(maxp_4)

# # Fully connected layers with Dropout
# dense_1 = Dense(256, activation='relu')(flatten)
# dropout_1 = Dropout(0.3)(dense_1)

# dense_2 = Dense(256, activation='relu')(dropout_1)
# dropout_2 = Dropout(0.3)(dense_2)

# # Output layers with appropriate activation functions
# output_1 = Dense(1, activation='sigmoid', name='gender_out')(dropout_1)
# output_2 = Dense(1, activation='linear', name='age_out')(dropout_2)

# # -----------------------------------------------------------
# model = Model(inputs=[inputs], outputs=[output_1, output_2])

# # Model Compilation with Adam optimizer and specified learning rate
# optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed
# model.compile(loss=['binary_crossentropy', 'mse'], optimizer=optimizer, metrics=['accuracy', 'accuracy'])

# # Model Fitting
# history = model.fit(x=x, y=[y_gender, y_age], batch_size=32, epochs=40, validation_split=0.2)
